## 1. 환경 설정

`(1) Env 환경변수`

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 기본 라이브러리`

In [2]:
import os
from glob import glob

from pprint import pprint

`(3) 벡터저장소 로드`  
- 저장해 둔 크로마 벡터저장소를 가져오기

In [3]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small", 
)

vectorstore = Chroma(
    embedding_function=embeddings,
    collection_name="chroma_test",
    persist_directory="./chroma_db",
    )

print(f"벡터 저장소에 저장된 문서 수: {vectorstore._collection.count()}")

벡터 저장소에 저장된 문서 수: 5


## 2. LangChain LCEL

### 2.1 Prompt + LLM

In [4]:
# 다중 메시지 전송
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# 모델 초기화
llm = ChatOpenAI(
    model="gpt-4o-mini", 
    temperature=0.3, 
    max_tokens=100,
    )

messages = [
    ("system", "You are a helpful assistant."),
    ("user", "{query}"),
]

# 메시지 리스트를 템플릿으로 변환
prompt = ChatPromptTemplate.from_messages(messages)

# 템플릿을 출력
print(prompt)

input_variables=['query'] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], template='{query}'))]


In [7]:
# 템플릿 입력 변수를 출력
print(prompt.input_variables)

['query']


In [8]:
# input 값을 전달하여 프롬프트를 렌더링
prompt_text = prompt.format(query="테슬라 창업자는 누구인가요?")

print(prompt_text)

System: You are a helpful assistant.
Human: 테슬라 창업자는 누구인가요?


In [9]:
# 모델에 prompt text를 직접 입력
response = llm.invoke(prompt_text)

# 모델의 응답을 출력
print(response.content)

테슬라의 창립자는 엘론 머스크(Elon Musk), 마틴 에버하르드(Martin Eberhard), 마크 타페닝(Mark Tarpenning), 제프 스프레처(JB Straubel), 이안 라이트(Ian Wright) 등입니다. 테슬라는 2003년에 설립되었으며, 엘론 머스크는 2004년에 투자자로 참여한 후 CEO로서 회사를 이끌고


In [10]:
# LCEL 체인을 구성
chain = prompt | llm

# 체인을 출력
print(chain)

first=ChatPromptTemplate(input_variables=['query'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], template='{query}'))]) last=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x1338c8710>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x133873dd0>, root_client=<openai.OpenAI object at 0x13388b790>, root_async_client=<openai.AsyncOpenAI object at 0x1338c8850>, model_name='gpt-4o-mini', temperature=0.3, openai_api_key=SecretStr('**********'), openai_proxy='', max_tokens=100)


In [11]:
# 체인의 입력 스키마를 출력
from pprint import pprint
pprint(chain.input_schema.schema())

{'properties': {'query': {'title': 'Query', 'type': 'string'}},
 'required': ['query'],
 'title': 'PromptInput',
 'type': 'object'}


In [12]:
# 체인을 실행 - 옵션 1
response = chain.invoke({"query":"테슬라 창업자는 누구인가요?"})

# 체인의 응답을 출력
print(response.content)

테슬라의 창립자는 엘론 머스크(Elon Musk)입니다. 그러나 테슬라는 2003년에 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Mark Tarpenning)에 의해 설립되었습니다. 엘론 머스크는 2004년에 투자자로 참여한 후, CEO로 취임하고 회사의 비전을 이끌어가면서 테슬라의 주요 인물로 자리잡게 되었습니다


In [13]:
# 체인을 실행 - 옵션 2
response = chain.invoke("테슬라 창업자는 누구인가요?")

# 체인의 응답을 출력
print(response.content)

테슬라의 창립자는 엘론 머스크(Elon Musk), 마틴 에버하드(Martin Eberhard), 마크 타페닝(Mark Tarpenning), 제프 스프레처(Jeffrey B. Straubel), 이안 라이트(Ian Wright) 등 여러 명이 있습니다. 그러나 엘론 머스크가 2004년에 회사에 투자하고 CEO로 취임한 이후, 테슬라의 얼굴이자 가장 잘


### 2.2 Prompt + LLM + Output Parser

`a) 문자열 파싱 - StrOutputParser`

In [14]:
response

AIMessage(content='테슬라의 창립자는 엘론 머스크(Elon Musk), 마틴 에버하드(Martin Eberhard), 마크 타페닝(Mark Tarpenning), 제프 스프레처(Jeffrey B. Straubel), 이안 라이트(Ian Wright) 등 여러 명이 있습니다. 그러나 엘론 머스크가 2004년에 회사에 투자하고 CEO로 취임한 이후, 테슬라의 얼굴이자 가장 잘', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 27, 'total_tokens': 127, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_483d39d857', 'finish_reason': 'length', 'logprobs': None}, id='run-71ae890c-1298-463f-9ea0-9200e151e8ac-0', usage_metadata={'input_tokens': 27, 'output_tokens': 100, 'total_tokens': 127})

In [15]:
# StrOutputParser - 문자열 출력을 파싱
from langchain_core.output_parsers import StrOutputParser

# 출력 파서를 생성
output_parser = StrOutputParser()

# 출력 파서를 실행
output_parser.invoke(response)

'테슬라의 창립자는 엘론 머스크(Elon Musk), 마틴 에버하드(Martin Eberhard), 마크 타페닝(Mark Tarpenning), 제프 스프레처(Jeffrey B. Straubel), 이안 라이트(Ian Wright) 등 여러 명이 있습니다. 그러나 엘론 머스크가 2004년에 회사에 투자하고 CEO로 취임한 이후, 테슬라의 얼굴이자 가장 잘'

In [16]:
str_chain = prompt | llm  | output_parser

query = "리비안의 설립년도는 언제인가요?"
str_response = str_chain.invoke(query)

print(str_response)



리비안(Rivian)은 2009년에 설립되었습니다. 이 회사는 전기차를 개발하고 생산하는 미국의 자동차 제조업체로, 특히 전기 픽업트럭과 SUV에 주력하고 있습니다.


`b) JSON 출력 - JsonOutputParser`

In [17]:
from langchain_core.output_parsers import JsonOutputParser

# 출력 파서를 생성
json_parser = JsonOutputParser()

# 체인을 실행 (JSON 출력)
json_response = chain.invoke("테슬라 창업자는 누구인가요? JSON 형식으로 출력해주세요.") 
print(json_response)

# 출력 파서를 실행
json_parser_output = json_parser.invoke(json_response)
print(json_parser_output)

content='```json\n{\n  "회사": "테슬라",\n  "창업자": [\n    {\n      "이름": "마틴 에버하드",\n      "역할": "공동 창립자"\n    },\n    {\n      "이름": "마크 타페닝",\n      "역할": "공동 창립자"\n    },\n    {\n      "이름": "엘론 머스크",\n      "역할": "투자자 및' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 34, 'total_tokens': 134, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_483d39d857', 'finish_reason': 'length', 'logprobs': None} id='run-24a84e04-6a83-4f37-b726-6305e6014d8e-0' usage_metadata={'input_tokens': 34, 'output_tokens': 100, 'total_tokens': 134}
{'회사': '테슬라', '창업자': [{'이름': '마틴 에버하드', '역할': '공동 창립자'}, {'이름': '마크 타페닝', '역할': '공동 창립자'}, {'이름': '엘론 머스크', '역할': '투자자 및'}]}


`c) Schema 지정 - PydanticOutputParser`

In [18]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field, validator

# Pydantic 모델을 생성
class Person(BaseModel):
    """Information about a person."""

    name: str = Field(..., description="The name of the person")
    title: str = Field(..., description="The title or position of the person.")

# 출력 파서를 생성
person_parser = PydanticOutputParser(pydantic_object=Person)
print("========================================")
print("PydanticOutputParser 프롬프트")
print("----------------------------------------")
print(person_parser.get_format_instructions())
print("========================================")


# Prompt 템플릿을 생성 - Pydantic 모델을 사용
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user query. Wrap the output in `json` tags\n{format_instructions}",
        ),
        ("human", "{query}"),
    ]
).partial(format_instructions=person_parser.get_format_instructions())

print("Prompt 템플릿")
print("----------------------------------------")
print(prompt.format(query="테슬라 창업자는 누구인가요?"))
print("========================================")


PydanticOutputParser 프롬프트
----------------------------------------
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "Information about a person.", "properties": {"name": {"title": "Name", "description": "The name of the person", "type": "string"}, "title": {"title": "Title", "description": "The title or position of the person.", "type": "string"}}, "required": ["name", "title"]}
```
Prompt 템플릿
----------------------------------------
System: Answer the user query. Wrap the output in `json` tags
The output should be formatted as a JSON instance that conforms to the JSON schema

In [19]:
# 체인을 구성
person_chain = prompt | llm | person_parser

# 체인을 실행
response = person_chain.invoke("테슬라 창업자는 누구인가요?")

# 체인의 응답을 출력
response

Person(name='엘론 머스크', title='CEO 및 창립자')

## 3. Chat Completion Methods

`(1) stream`  
- 입력에 대한 응답을 실시간 스트림을 생성하여 전달

In [21]:
import time 

for chunk in llm.stream("테슬라 창업자는 누구인가요?"):
    # 기본적으로 print 함수는 출력을 할 때마다 줄바꿈을 하지만, 줄바꿈 없이 출력하려면 end=""를 사용하면 됩니다.
    # flush=True 옵션을 사용하여 출력 버퍼를 즉시 비웁니다. 데이터를 지연 없이 즉시 출력하는 데 유용합니다.
    print(chunk.content, end="", flush=True)  
    # time.sleep(0.1)  # 0.1초 대기 (100ms)

테슬라의 창립자는 엘론 머스크(Elon Musk)입니다. 그러나 테슬라는 2003년에 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Mark Tarpenning)에 의해 설립되었습니다. 엘론 머스크는 2004년에 테슬라에 투자하고 이후 CEO로 취임하면서 회사의 성장에 큰 영향을 미쳤습니다.

`(2) batch`  
- 입력 리스트에 대한 응답을 배치 단위로 생성

In [22]:
questions = [
    "테슬라의 창업자는 누구인가요?",
    "리비안의 창업자는 누구인가요?",
]

responses = llm.batch(questions)

for response in responses:
    response.pretty_print()
    print()

================================== Ai Message ==================================

테슬라의 창립자는 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Mark Tarpenning)입니다. 이 두 사람은 2003년에 테슬라 모터스를 설립했습니다. 이후 일론 머스크(Elon Musk)가 2004년에 투자자로 참여하면서 회사의 주요 인물 중 하나가 되었고, 이후 CEO로 취임하여 회사의 성장에 큰 영향을 미쳤습니다.

================================== Ai Message ==================================

리비안(Rivian)의 창업자는 RJ 스칼링(RJ Scaringe)입니다. 그는 2009년에 리비안을 설립하였으며, 전기차 및 친환경 차량 개발에 주력하고 있습니다. 리비안은 전기 픽업트럭인 R1T와 전기 SUV인 R1S를 출시하여 주목받고 있습니다.



## 4. Runnable

`(1) RunnableParallel`

In [23]:
# 문서 검색기 생성
retriever = vectorstore.as_retriever(
    search_kwargs={'k': 1}, 
)

query = "테슬라 창업자는 누구인가요?"
retrieved_docs = retriever.invoke(query)

retrieved_docs_text = "\n".join([doc.page_content for doc in retrieved_docs])

pprint(retrieved_docs_text)

('테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 '
 '마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다. '
 '머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 '
 '이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다.')


In [24]:
from langchain_core.runnables import RunnableParallel
from operator import itemgetter

# RunnableParrellel 구성
runnable = RunnableParallel(
    {"context": itemgetter("context") , "question": itemgetter("question")}
)

# 객체를 실행
response = runnable.invoke({"context": retrieved_docs_text, "question": query})

# 응답을 출력
response

{'context': '테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다. 머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다.',
 'question': '테슬라 창업자는 누구인가요?'}

`(2) RunnablePassthrough`

In [26]:
from langchain_core.runnables import RunnablePassthrough

runnable = RunnableParallel(
    question=RunnablePassthrough(),
)

runnable.invoke({"query":"테슬라 창업자는 누구인가요?"})

{'question': {'query': '테슬라 창업자는 누구인가요?'}}

`(3) RunnableLambda`
- 정의: 파이썬의 커스텀 함수를 매핑하는데 사용

In [27]:
from langchain_core.runnables import RunnableLambda

def count_num_words(text):
    return len(text.split())

runnable = RunnableParallel(
    question=RunnablePassthrough(),
    word_count=RunnableLambda(count_num_words),
)

runnable.invoke("테슬라 창업자는 누구인가요?") 

{'question': '테슬라 창업자는 누구인가요?', 'word_count': 3}

## 5. 전체 RAG 파이프라인 구성

`(1) RAG 프롬프트 템플릿`  

In [28]:
# Prompt 템플릿을 생성
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context.
Do not use any external information or knowledge. 
If the answer is not in the context, answer "잘 모르겠습니다.".

[Context]
{context}

[Question] 
{question}

[Answer]
"""

prompt = ChatPromptTemplate.from_template(template)

# 템플릿을 출력
prompt.pretty_print()

================================ Human Message =================================

Answer the question based only on the following context.
Do not use any external information or knowledge. 
If the answer is not in the context, answer "잘 모르겠습니다.".

[Context]
{context}

[Question] 
{question}

[Answer]



`(2) Retriever Chain 연결`  

In [29]:
# 벡터 검색기
retriever = vectorstore.as_retriever(search_kwargs={'k': 2})

# 문서 포맷터 함수
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

# 체인 구성
retriever_chain = retriever | format_docs

# 체인을 실행
response = retriever_chain.invoke("테슬라 창업자는 누구인가요?")

pprint(response)

('테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 '
 '마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다. '
 '머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 '
 '이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다.\n'
 '\n'
 '2023년 테슬라는 1,808,581대의 차량을 판매하여 2022년에 비해 37.65% 증가했습니다. 2012년부터 2023년 3분기까지 '
 '테슬라의 전 세계 누적 판매량은 4,962,975대를 초과했습니다. SMT Packaging에 따르면, 2023년 테슬라의 판매량은 전 '
 '세계 전기차 시장의 약 12.9%를 차지했습니다.')


`(3) RAG Chain 연결`  

In [30]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI


# LLM 모델 생성
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, max_tokens=100)

# 체인 생성
rag_chain = (
    {"context": retriever_chain , "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 체인 실행
query = "테슬라 창업자는 누구인가요?"
response = rag_chain.invoke(query)

In [31]:
# 결과 출력
response

'마틴 에버하드와 마크 타페닝입니다.'

## 6. Gradio 챗봇

`(1) invoke 실행` 

In [32]:
import gradio as gr

def answer_invoke(message, history):
    response = rag_chain.invoke(message)
    return response

# Graiio 인터페이스 생성 
demo = gr.ChatInterface(fn=answer_invoke, title="QA Bot")

# Graiio 실행  
demo.launch()

/Users/steve2/Library/Caches/pypoetry/virtualenvs/langchain-env-9sqiXrer-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [33]:
# Graiio 종료
demo.close()

Closing server running on port: 7860


`(2) stream 실행` 

In [34]:
import gradio as gr

def answer_invoke(message, history):
    partial_message = ""
    for chunk in rag_chain.stream(message):
        if chunk is not None:
            partial_message = partial_message + chunk
            time.sleep(0.1)
            yield partial_message

# Graiio 인터페이스 생성 
demo = gr.ChatInterface(fn=answer_invoke, title="QA Bot")

# Graiio 실행  
demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [35]:
# Graiio 종료
demo.close()

Closing server running on port: 7860
